# Questrade Portfolio Overview

In [ ]:
import os
import math
import ratios
import requests
import momentum
import pyticker
import dividend
import questrade
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
from qtrade import Questrade as qt
import pandas_datareader.data as web
#from questrade_api import Questrade

## Account Balance Summary

In [ ]:
balance = questrade.get_balance()
balance

## Position & Performance Summary

In [ ]:
portfolio = questrade.get_positions()

In [ ]:
portfolio['%Portfolio'].plot(kind='pie',
                             title='Portfolio Breakdown',
                             figsize=(10, 10),
                             legend=True)

In [ ]:
portfolio = portfolio.style.applymap(questrade.color_negative_red, subset=['Gain/Loss (%)'])
portfolio

## Monthly Dividend Income

In [ ]:
div = questrade.get_dividend_income()
div.loc['Total'] = div.sum(numeric_only=True, axis=0)
div

In [ ]:
div[:-1].plot(kind='bar',
                    title='Monthly Dividend Payments',
                    xlabel='Months',
                    ylabel='Dividend ($)',
                    figsize=(15, 10),
                    legend=False,
                    ylim=(0, 30))

## Global Macro Momentum

In [ ]:
global_macro = ['SPY', 'QQQ', 'TLT', 'IEF', 'GLD', 'DBC']
period = [1,3,6,12,24,36,48,60]

data = {'Symbol': [], 'Momentum': []}
count = 0
for symbol in global_macro:
    count += 1
    print(f"{count}/{len(global_macro)}")
    data['Symbol'].append(symbol)
    data['Momentum'].append(momentum.calculate_equal_weight_momentum(symbol, period))
momentum_df = pd.DataFrame(data)
momentum_df.set_index('Symbol')
momentum_df.sort_values(by='Momentum',inplace=True, ascending=False)
momentum_df

## US Sector Momentum

In [ ]:
sector_df = pyticker.get_sector_df()
period = [1,3,6,12]
count = 0
for x in list(sector_df.index):
    count += 1
    print(f"{count}/{len(list(sector_df.index))}")
    sector_df.loc[x,'Momentum'] = momentum.calculate_equal_weight_momentum(x, period)

In [ ]:
sector_df.sort_values(by='Momentum', inplace=True, ascending=False)
sector_df

## Dow Jones Equity Momentum

In [ ]:
symbols = pyticker.get_symbols_by_index('DOW JONES')

data = {'Symbol': [], 'Momentum': []}
count = 0
for symbol in symbols:
    count += 1
    print(f"{count}/{len(symbols)}")
    data['Symbol'].append(symbol)
    try:
        data['Momentum'].append(momentum.calculate_equal_weight_momentum(symbol, [1,3,6,12,24,36]))
    except:
        data['Momentum'].append(0)

df = pd.DataFrame(data)
df.set_index('Symbol', inplace=True)
df

In [ ]:
df = df.sort_values(by='Momentum', ascending=False)
df

In [ ]:
top_ten = df.head(10)
top_ten

In [ ]:
data = {'Symbol': [], 'Revenue_Growth': [], 'ROE': [], 'Dividend_Growth': [], 'Dividend_Yield': []}

PERIOD = 20

for symbol in top_ten.index:
    print(symbol)
    fr = ratios.get_financial_ratios(symbol)
    data['Symbol'].append(symbol)
    data['Revenue_Growth'].append(fr.loc['revenueGrowth','Value'])
    data['ROE'].append(fr.loc['returnOnEquityTTM','Value'])
    data['Dividend_Growth'].append(dividend.calcualte_avg_dividend_growth(symbol,20))
    data['Dividend_Yield'].append(dividend.calculate_current_dividend_yield(symbol))

data

In [ ]:
ratio_df = pd.DataFrame(data)
ratio_df.set_index('Symbol', inplace=True)
ratio_df = ratio_df[(ratio_df['Revenue_Growth'] > 0.02) & (ratio_df['ROE'] > 0.10)]
ratio_df